In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from pprint import pprint
pd.set_option('display.width', None)
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def drop_by_index(X,indexes):
    """
    helper function to drop rows of dataframe and return new dataframe without those rows with indexes resetted
    """
    X = X.drop(indexes)
    X = X.reset_index().drop(columns="index")
    return(X)

### Data Processing

In [ ]:
clinical = pd.read_pickle("C:\\SMU_v2\\clinical_output.pkl").reset_index().drop(columns="index")
to_drop = clinical[clinical['dx_date']=="NA"].index
clinical = drop_by_index(clinical,to_drop)

clinical.drop(columns=['NRIC','dob',\
                       'Count_as_DFS','Count_as_OS','Count_as_CSS'],inplace = True)

# drop all rows where dates are null
    # wave 1 details
clinical.dropna(axis=0, \
                subset=['Date_for_DFS','Date_for_OS','Date_for_CSS','dx_date',\
                        'size_precise', 'nodespos'],\
                inplace=True)
# wave 2 details


# convert all datetime in dataframe into dateime format for processing
clinical["Date_for_DFS"] = pd.to_datetime(clinical["Date_for_DFS"])
clinical["Date_for_OS"] = pd.to_datetime(clinical["Date_for_OS"])
clinical["Date_for_CSS"] = pd.to_datetime(clinical["Date_for_CSS"])
clinical["dx_date"] = pd.to_datetime(clinical["dx_date"])
clinical['last_seen']= pd.to_datetime(clinical["dx_date"])


# calculate in days
clinical["DFS_days"] = (clinical["Date_for_DFS"] - clinical['dx_date'] )/np.timedelta64(1, 'D')
clinical["OS_days"] = (clinical["Date_for_OS"] - clinical['dx_date'] )/np.timedelta64(1, 'D')
clinical["CSS_days"] = (clinical["Date_for_CSS"] - clinical['dx_date'] )/np.timedelta64(1, 'D')

# alive or dead
clinical['status'] = np.where(clinical['death_age'].isnull(), False, True)

In [ ]:
# clinical.head(5)
# clinical.to_csv("C:\\Users\\LINGXING\\Desktop\\clinical.csv",index = False)

### Data filtering

Data of our interest are 5 and 10 years, patient that are new (does not have sufficient records will disturb and mess up our accuracy level

In [ ]:
'''
df_dict[] : 5 years | 10 years data
df_dict[][] : years, survival type
'''
df_dict = {}
  
#years that we are interested in
for i in list([1,2,5,10]):
    tmp = {}
    for x in list(["DFS", "CSS", "OS"]):
        clinical['{}_{}_years'.format(x, i)] = np.where(
                                                         np.logical_or(clinical['death_age'] > 0,\
                                                         clinical['{}_days'.format(x)]/(365.25*i) >= i),\
                                                True,False)
        tmp[x] = clinical[clinical['{}_{}_years'.format(x, i)] == True]
    df_dict['{}_years'.format(i)] = tmp

### Display shape of data after filtering

In [ ]:
for i in df_dict:
    for s_type in df_dict[i]:
        print("Year: {}, survival type: {}, size: {}".format(i,s_type,df_dict[i][s_type].shape))

In [ ]:
from sksurv.nonparametric import kaplan_meier_estimator

YEAR = 5
STYPE = "OS"

time, survival_prob = kaplan_meier_estimator(df_dict['{}_years'.format(YEAR)][STYPE]['status'], 
                                             df_dict['{}_years'.format(YEAR)][STYPE]['OS_days'])
time = time/365.25
# print(max(time),min(time))
plt.step(time, survival_prob, where="post")
plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$ (Days)")
plt.title("{} Years Survival Rate for {}".format(YEAR,STYPE))
plt.grid(True)

In [ ]:
Years = 5

for survival in ("DFS", "OS", 'CSS'):
    time_treatment, survival_prob_treatment = kaplan_meier_estimator(
        df_dict['{}_years'.format(YEAR)][survival]['status'],
        df_dict['{}_years'.format(YEAR)][survival]['{}_days'.format(survival)])
    
    plt.step(time_treatment, survival_prob_treatment, where="post",
             label="Survival Type = {}".format(survival))

plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.title("{} Years Surivial Rate For Each Category".format(YEAR))
plt.grid(True)
plt.legend(loc="best")

In [ ]:
plt.rcParams["figure.figsize"] = (30,10)
for value in df_dict['{}_years'.format(YEAR)]['OS']["TNM_Stage"].unique():
    mask = df_dict['{}_years'.format(YEAR)]['OS']["TNM_Stage"] == value
    time_cell, survival_prob_cell = kaplan_meier_estimator(df_dict['{}_years'.format(YEAR)][STYPE]['status'][mask],
                                                           df_dict['{}_years'.format(YEAR)][STYPE]['{}_days'.format(STYPE)][mask])
    plt.step(time_cell, survival_prob_cell, where="post",
             label= '{} (n = {})'.format(value, mask.sum()))

plt.ylabel("est. probability of survival $\hat(t)$")
plt.xlabel("time $t$")
plt.grid(True)
plt.legend(loc="best")

### Selecting population pool

Survival times are subject to right-censoring, therefore, we need to consider an individual's status
in addition to survival time. To be fully compatible with scikit-learn, Status and Survival_in_days
need to be stored as a structured array with the first field indicating whether the actual survival time
was observed or if was censored, and the second field denoting the observerd survival time,
which corresponds to the time of death (if Status == 'dead', $\delta = 1$) or the last time that
person was contacted (if Status == 'alive', $\delta = 0$).


In [ ]:
'''
drop: cerbB2, ProgStage_AJCC8, c_Staging, p_Staging, TNM_Stage, 'cause_of_death'
'''
YEAR = 5
DF_TO_USE = df_dict['{}_years'.format(YEAR)][STYPE]


X = DF_TO_USE[['Gender', 'c_tstage', 'cNstage', 'cMstage', 'tstage',\
              'nstage', 'Mstage', 'diff', 'ER', 'PR',\
              'Her2', 'size_precise', 'nodespos', 'Age_@_Dx']]

Y = DF_TO_USE[['status','OS_days']]

# convert to int since some fields fro age_@_dx is null
X.loc[:,"Age_@_Dx"] = X["Age_@_Dx"].astype("int16")

# OHE for probability
from sksurv.preprocessing import OneHotEncoder
X = OneHotEncoder().fit_transform(X)

# convert Y to structured array
s = Y.dtypes
Y = np.array([tuple(x) for x in Y.values], dtype=list(zip(s.index, s)))

print("X shape: {}".format(X.shape))
print("Y shape: {}".format(Y.shape))

### Calculation matrix for CoxPHSurvivalAnalysis - Cox's proportional hazard's model

tol is like the p value
|1 - (new neg. log-likelihood / old neg. log-likelihood) | < tol

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,  test_size=0.33, random_state=42)

from sksurv.linear_model import CoxPHSurvivalAnalysis

# since features are highly corelated, reducing alpha values to smaller values allows the learning
estimator = CoxPHSurvivalAnalysis(alpha = 1e-4, verbose = 1)
estimator.fit(X_train, Y_train)

# X.isnull().sum()
# X_train.head()

### Log Hazarad Ratio
In survival analysis, the hazard ratio (HR) is the ratio of the hazard rates corresponding to the conditions described by two levels of an explanatory variable. For example, in a drug study, the treated population may die at twice the rate per unit time as the control population. The hazard ratio would be 2, indicating higher hazard of death from the treatment. Or in another study, men receiving the same treatment may suffer a certain complication ten times more frequently per unit time than women, giving a hazard ratio of 10. - wiki

In [ ]:
tmp = pd.Series(estimator.coef_, index=X.columns)
tmp = tmp.to_frame("Log Hazarad Ratio")
tmp = tmp.sort_values(by=['Log Hazarad Ratio'])
tmp

### Measuring the Performance of Survival Models
Our test data is usually subject to censoring (only verified records are available, events happening in between is skipped), therefore metrics like root mean squared error or correlation are unsuitable. Instead, we use generalization of the area under the receiver operating characteristic (ROC) curve called Harrell's concordance index or c-index.

The interpretation is identical to the traditional area under the ROC curve metric for binary classification:

- a value of 0.5 denotes a random model,
- a value of 1.0 denotes a perfect model,
- a value of 0.0 denotes a perfectly wrong model.

In [ ]:
from sksurv.metrics import concordance_index_censored

prediction = estimator.predict(X_test)
result = concordance_index_censored(Y_test["status"], Y_test["OS_days"], prediction)
result[0]

### Feature Selection

In [ ]:
def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = CoxPHSurvivalAnalysis(alpha = 1e-4)
    for j in range(n_features):
        Xj = X[:, j:j+1]
        m.fit(Xj, y)
        scores[j] = m.score(Xj, y)
    return scores

scores = fit_and_score_features(X_test.values, Y_test)
pd.Series(scores, index=X_test.columns).sort_values(ascending=False)

### Determine features that are useful

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline

pipe = Pipeline([('encode', OneHotEncoder()),
                 ('select', SelectKBest(fit_and_score_features, k=3)),
                 ('model', CoxPHSurvivalAnalysis(alpha = 1e-6, tol= 1e-6))])

from sklearn.model_selection import GridSearchCV

param_grid = {'select__k': np.arange(1, X.shape[1] + 1)}
gcv = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, iid=True)
gcv.fit(X, Y)

pd.DataFrame(gcv.cv_results_).sort_values(by='mean_test_score', ascending=False)

In [ ]:
pipe.set_params(**gcv.best_params_)
pipe.fit(X, Y)

encoder, transformer, final_estimator = [s[1] for s in pipe.steps]
pd.Series(final_estimator.coef_, index=encoder.encoded_columns_[transformer.get_support()])

### RandomSurvivalForest

In [ ]:
from sksurv.ensemble import RandomSurvivalForest

random_state = 20
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=random_state)
rsf.fit(X_train, Y_train)

In [ ]:
rsf.score(X_test, Y_test)

### Predicting

For prediction, a sample is dropped down each tree in the forest until it reaches a terminal node. Data in each terminal is used to non-parametrically estimate the survival and cumulative hazard function using the Kaplan-Meier and Nelson-Aalen estimator, respectively. In addition, a risk score can be computed that represents the expected number of events for one particular terminal node. The ensemble prediction is simply the average across all trees in the forest.

In [ ]:
X_test_values = X_test.values
a = np.empty(X_test.shape[0], dtype=[("Age_@_Dx", float), ("nodespos", float)])
a["Age_@_Dx"] = X_test_values[:, -1]
a["nodespos"] = X_test_values[:, -2]

sort_idx = np.argsort(a, order=["nodespos", "Age_@_Dx"])

X_test_sel = pd.DataFrame(
    X_test_values[np.concatenate((sort_idx[:3], sort_idx[-3:]))],
    columns=list(X_test.columns))

X_test_sel

In [ ]:
#predict risk score
pd.Series(rsf.predict(X_test_sel))

In [ ]:
surv = rsf.predict_survival_function(X_test_sel)

for i, s in enumerate(surv):
    plt.step(rsf.event_times_, s, where="post", label=str(i))
plt.ylabel("Survival probability")
plt.xlabel("Time in days")
plt.grid(True)
plt.legend()

In [ ]:
surv = rsf.predict_cumulative_hazard_function(X_test_sel)

for i, s in enumerate(surv):
    plt.step(rsf.event_times_, s, where="post", label=str(i))
plt.ylabel("Cumulative hazard")
plt.xlabel("Time in days")
plt.grid(True)
plt.legend()

### Permutation-based Feature Importance

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

feature_names = X.columns.tolist()
perm = PermutationImportance(rsf, n_iter=15, random_state=random_state)
perm.fit(X_test, Y_test)
eli5.show_weights(perm, feature_names=feature_names)

### Testing

In [ ]:
# new_X_features = X_features.dropna(axis = 0, how ='any')  
# print("Old data frame length:", len(X_features)) 
# print("New data frame length:", len(new_X_features)) 
# print("Number of rows with at least 1 NA value: ", (len(X_features)-len(new_X_features))) 

In [ ]:
hyperparams = {
    'L2_reg': 10.0,
    'batch_norm': True,
    'dropout': 0.4,
    'hidden_layers_sizes': [25, 25],
    'learning_rate': 1e-05,
    'lr_decay': 0.001,
    'momentum': 0.9,
    'n_in': train_data['x'].shape[1],
    'standardize': True
}

### Limitation: Not enough data per group

In [ ]:
# X_features = clinical[['NRIC','Gender', 'c_tstage', 'cNstage', 'cMstage', 'c_Staging', 'tstage',\
#                       'nstage', 'Mstage', 'p_Staging', 'diff', 'TNM_Stage', 'ProgStage_AJCC8',\
#                       'ER', 'PR', 'cerbB2', 'Her2', 'size_precise', 'nodespos', 'Age_@_Dx']]

# X_features['COUNTER'] =1  
# grouped_data = X_features.groupby(['TNM_Stage','ER', 'PR','Her2']).agg({'COUNTER': ['count']})
# grouped_data = grouped_data.reset_index()
# df = pd.DataFrame(data=grouped_data)

thought process
1. calculate the years of DFS, OS, CSS
2. group the data into disease features. calculate the the mean and sd
3. caculate the z score and probility area
4. split the df into x and y. X = clinical features, Y = years DFS, years CSS, years OS, z score, probility area and SD for modelling category

In [ ]:
# from scipy.stats import zscore
# ''' 
#     calculate avg death for dfs, os, css
# ''' 

# # get all the death records
# death_clinical = clinical[clinical['death_age'].notnull()] # shape: (5910, 33)

# # get avg for DFS, OS, CSS for those who has deceased
# dfs_mean = death_clinical['DFS_years'].mean(axis = 0)
# os_mean = death_clinical['OS_years'].mean(axis = 0)
# css_mean = death_clinical['CSS_years'].mean(axis = 0)

# # get sd for DFS, OS, CSS for those who has deceased
# dfs_std = death_clinical['DFS_years'].std(axis = 0,ddof=1)
# os_std = death_clinical['OS_years'].std(axis = 0,ddof=1)
# css_std = death_clinical['CSS_years'].std(axis = 0,ddof=1)

# print("dfs_mean: {}, os_mean: {}, css_mean: {}".format(dfs_mean,os_mean,css_mean))
# print("dfs_std: {}, os_std: {}, css_std: {}".format(dfs_std,os_std,css_std))

# for x in ['DFS_years', 'OS_years','CSS_years']:
#     death_clinical["{}_zscore".format(x)] = death_clinical[[x]].apply(zscore)

# death_clinical.head(10)
